# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

In this Colab, we present a step-by-step guide on how to fine-tune Whisper
for any multilingual ASR dataset using Hugging Face 🤗 Transformers. This is a
more "hands-on" version of the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper).
For a more in-depth explanation of Whisper, the Common Voice dataset and the theory behind fine-tuning, the reader is advised to refer to the blog post.

## Introduction

Whisper is a pre-trained model for automatic speech recognition (ASR)
published in [September 2022](https://openai.com/blog/whisper/) by the authors
Alec Radford et al. from OpenAI. Unlike many of its predecessors, such as
[Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), which are pre-trained
on un-labelled audio data, Whisper is pre-trained on a vast quantity of
**labelled** audio-transcription data, 680,000 hours to be precise.
This is an order of magnitude more data than the un-labelled audio data used
to train Wav2Vec 2.0 (60,000 hours). What is more, 117,000 hours of this
pre-training data is multilingual ASR data. This results in checkpoints
that can be applied to over 96 languages, many of which are considered
_low-resource_.

When scaled to 680,000 hours of labelled pre-training data, Whisper models
demonstrate a strong ability to generalise to many datasets and domains.
The pre-trained checkpoints achieve competitive results to state-of-the-art
ASR systems, with near 3% word error rate (WER) on the test-clean subset of
LibriSpeech ASR and a new state-of-the-art on TED-LIUM with 4.7% WER (_c.f._
Table 8 of the [Whisper paper](https://cdn.openai.com/papers/whisper.pdf)).
The extensive multilingual ASR knowledge acquired by Whisper during pre-training
can be leveraged for other low-resource languages; through fine-tuning, the
pre-trained checkpoints can be adapted for specific datasets and languages
to further improve upon these results. We'll show just how Whisper can be fine-tuned
for low-resource languages in this Colab.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 1:</b> Whisper model. The architecture
follows the standard Transformer-based encoder-decoder model. A
log-Mel spectrogram is input to the encoder. The last encoder
hidden states are input to the decoder via cross-attention mechanisms. The
decoder autoregressively predicts text tokens, jointly conditional on the
encoder hidden states and previously predicted tokens. Figure source:
<a href="https://openai.com/blog/whisper/">OpenAI Whisper Blog</a>.</figcaption>
</figure>

The Whisper checkpoints come in five configurations of varying model sizes.
The smallest four are trained on either English-only or multilingual data.
The largest checkpoints are multilingual only. All 11 of the pre-trained checkpoints
are available on the [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). The
checkpoints are summarised in the following table with links to the models on the Hub:

| Size     | Layers | Width | Heads | Parameters | English-only                                         | Multilingual                                        |
|----------|--------|-------|-------|------------|------------------------------------------------------|-----------------------------------------------------|
| tiny     | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)    |
| base     | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)     |
| small    | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)    |
| medium   | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium)   |
| large    | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)    |
| large-v2 | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v2) |
| large-v3 | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v3) |


For demonstration purposes, we'll fine-tune the multilingual version of the
[`"small"`](https://huggingface.co/openai/whisper-small) checkpoint with 244M params (~= 1GB).
As for our data, we'll train and evaluate our system on a low-resource language
taken from the [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0)
dataset. We'll show that with as little as 8 hours of fine-tuning data, we can achieve
strong performance in this language.

------------------------------------------------------------------------

\\({}^1\\) The name Whisper follows from the acronym “WSPSR”, which stands for “Web-scale Supervised Pre-training for Speech Recognition”.

## Prepare Environment

First of all, let's try to secure a decent GPU for our Colab! Unfortunately, it's becoming much harder to get access to a good GPU with the free version of Google Colab. However, with Google Colab Pro one should have no issues in being allocated a V100 or P100 GPU.

To get a GPU, click _Runtime_ -> _Change runtime type_, then change _Hardware accelerator_ from _CPU_ to one of the available GPUs, e.g. _T4_ (or better if you have one available). Next, click `Connect T4` in the top right-hand corner of your screen (or `Connect {V100, A100}` if you selected a different GPU).

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets[audio]` to download and prepare our training data, alongside
`transformers` and `accelerate` to load and train our Whisper model.
We'll also require the `soundfile` package to pre-process audio files,
`evaluate` and `jiwer` to assess the performance of our model, and
`tensorboard` to log our metrics. Finally, we'll use `gradio` to build a
flashy demo of our fine-tuned model.

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!

Linking the notebook to the Hub is straightforward - it simply requires entering your
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [ ]:
import os

current_directory = os.getcwd()
print(current_directory)

Using 🤗 Datasets, downloading and preparing data is extremely simple.
We can download and prepare the Common Voice splits in just one line of code.

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

Since Hindi is very low-resource, we'll combine the `train` and `validation`
splits to give approximately 8 hours of training data. We'll use the 4 hours
of `test` data as our held-out test set:

In [2]:
from datasets import load_dataset, DatasetDict,Audio



# common_voice = load_dataset(
#     path="/home/ubuntu/Downloads/voice-en/cv-corpus-19.0-delta-2024-09-13/en",
#     data_files={"train": "validated.tsv"}
# )
# # Split the dataset into train and test (for example, 80% train, 20% test)
# splits = common_voice["train"].train_test_split(test_size=0.2)  # seed for reproducibility

# # Create the DatasetDict with the splits
# common_voice = DatasetDict({
#     'train': splits['train'],
#     'test': splits['test']
# })



# common_voice = load_dataset(
#     path="/home/ubuntu/Documents/cv-corpus-19.0-delta-2024-09-13/en",
#     data_files={"train":'test1.tsv'}
# )

# # Split the dataset into train and test (e.g., 80% train, 20% test)
# splits = common_voice["train"].train_test_split(test_size=0.2)  # seed for reproducibility

# # Create the DatasetDict with the splits
# common_voice = DatasetDict({
#     'train': splits['train'],
#     'test': splits['test']
# })

common_voice = load_dataset(
    'csv',  # Use csv loader for TSV files
    data_files='/home/ubuntu/Documents/cv-corpus-19.0-delta-2024-09-13/en/test6.tsv',  # Your TSV file path
    delimiter='\t',  # Specify tab delimiter for TSV
    cache_dir=None  # Optional: disable caching if needed
)



# Then create your train/test split
splits = common_voice['train'].train_test_split(
    test_size=0.2,
    seed=40  # for reproducibility
)

# Create the final DatasetDict
common_voice = DatasetDict({
    'train': splits['train'],
    'test': splits['test']
})

common_voice = common_voice.cast_column("path", Audio(sampling_rate=16000))
# Print the dataset structure
print(common_voice)

# Print the column names
print("Available columns:", common_voice["train"].column_names)
print("Available columns (test):", common_voice["test"].column_names)

print("First training sample:", common_voice["train"][0])
print("First test sample:", common_voice["test"][0])




DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment'],
        num_rows: 33
    })
    test: Dataset({
        features: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment'],
        num_rows: 9
    })
})
Available columns: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment']
Available columns (test): ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment']
First training sample: {'client_id': '0e0501c09d84645ee7418e6bebbb87d59030031307c156e58c08f4ac5c00863e92c0b86f081386ae11eedea11cd0832ee23f7e50285fac52d39c09ed7851fd6d', 'path': {'path': '/

In [3]:

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


print("First training sample:", common_voice["train"][0])
print("First test sample:", common_voice["test"][0])

First training sample: {'client_id': '0e0501c09d84645ee7418e6bebbb87d59030031307c156e58c08f4ac5c00863e92c0b86f081386ae11eedea11cd0832ee23f7e50285fac52d39c09ed7851fd6d', 'path': {'path': '/home/ubuntu/Documents/cv-corpus-19.0-delta-2024-09-13/en/drone_command_ks/drone_ks_10.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        1.10094334e-09, -6.10725692e-08, -6.46566036e-08]), 'sampling_rate': 16000}, 'sentence_id': 'f05b13340f4242fbc3911588673c38d8d065cde77b605d63e85d3dcdf4931e02', 'sentence': 'Drone Alpha turn left 180 degree.  ', 'sentence_domain': None, 'up_votes': 2, 'down_votes': 0, 'age': None, 'gender': None, 'accents': 'United States English', 'variant': None, 'locale': 'en', 'segment': None}
First test sample: {'client_id': '0e0501c09d84645ee7418e6bebbb87d59030031307c156e58c08f4ac5c00863e92c0b86f081386ae11eedea11cd0832ee23f7e50285fac52d39c09ed7851fd6d', 'path': {'path': '/home/ubuntu/Documents/cv-corpus-19.0-delta-2024-09-13/en/drone_command_ks

Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [4]:
print(common_voice.column_names)

# columns_to_remove = ["sentence_id", "sentence_domain", "variant"]  # Adjust based on your dataset
# common_voice = common_voice.remove_columns(columns_to_remove)

common_voice = common_voice.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes","variant"])

print(common_voice)


{'train': ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment'], 'test': ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment']}
DatasetDict({
    train: Dataset({
        features: ['path', 'sentence_id', 'sentence', 'sentence_domain'],
        num_rows: 33
    })
    test: Dataset({
        features: ['path', 'sentence_id', 'sentence', 'sentence_domain'],
        num_rows: 9
    })
})


## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages:

1. A feature extractor which pre-processes the raw audio-inputs
2. The model which performs the sequence-to-sequence mapping
3. A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Hindi, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to
specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences:

In [6]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="English", task="transcribe")

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="English", task="transcribe")

### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [8]:
print(common_voice["train"])

Dataset({
    features: ['path', 'sentence_id', 'sentence', 'sentence_domain'],
    num_rows: 33
})


Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [9]:
from datasets import Audio

common_voice = common_voice.cast_column("path", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [10]:
print(common_voice["train"].column_names)

['path', 'sentence_id', 'sentence', 'sentence_domain']


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [11]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]

#     # compute log-Mel input features from input audio array
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

#     # encode target text to label ids
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch

# # Print available columns
# print("Available columns:", common_voice.column_names["train"])
# print(common_voice["train"][0])

import librosa
import soundfile as sf

def prepare_dataset(batch):
    # Extract the audio array and sampling rate from the 'path' dictionary
    audio_array = batch["path"]["array"]
    sampling_rate = batch["path"]["sampling_rate"]

    # Compute log-Mel input features from the audio array
    batch["input_features"] = feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0]

    # Encode the target text to label IDs
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

print("Available columns:", common_voice.column_names["train"])
print(common_voice["train"][0])

Available columns: ['path', 'sentence_id', 'sentence', 'sentence_domain']
{'path': {'path': '/home/ubuntu/Documents/cv-corpus-19.0-delta-2024-09-13/en/drone_command_ks/drone_ks_10.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        1.10094334e-09, -6.10725692e-08, -6.46566036e-08]), 'sampling_rate': 16000}, 'sentence_id': 'f05b13340f4242fbc3911588673c38d8d065cde77b605d63e85d3dcdf4931e02', 'sentence': 'Drone Alpha turn left 180 degree.  ', 'sentence_domain': None}


We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [12]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it
to transcribe speech in Hindi.

### Load a Pre-Trained Checkpoint

We'll start our fine-tuning run from the pre-trained Whisper `small` checkpoint,
the weights for which we need to load from the Hugging Face Hub. Again, this
is trivial through use of 🤗 Transformers!

In [13]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

We can disable the automatic language detection task performed during inference, and force the model to generate in Hindi. To do so, we set the [langauge](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.language)
and [task](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.task)
arguments to the generation config. We'll also set any [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)
to None, since this was the legacy way of setting the language and
task arguments:

In [14]:
model.generation_config.language = "english"
# model.generation_config.language = "<|en|>"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [15]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [16]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [17]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [18]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
from transformers import Seq2SeqTrainingArguments

# training_args = Seq2SeqTrainingArguments(
#     output_dir="/home/ubuntu/Downloads/whisper-small-en",  # change to a repo name of your choice
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
#     learning_rate=1e-5,
#     warmup_steps=500,
#     max_steps=4000,
#     gradient_checkpointing=True,
#     fp16=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     generation_max_length=225,
#     save_steps=1000,
#     eval_steps=1000,
#     logging_steps=25,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=True,
#     remove_unused_columns=True 
# )




training_args = Seq2SeqTrainingArguments(
    output_dir="/home/ubuntu/Downloads/drone_ks_tiny_t6V2",
    
    # Increase learning rate to fit your voice patterns more aggressively
    learning_rate=3e-5,

    warmup_ratio=0.1, 
    
    # Remove regularization since we want overfitting
      weight_decay=0.003,
    
    # Batch size can be kept small since dataset is specific
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    
    # More training steps to really learn your voice
    max_steps=1000,
    
    # Frequent evaluation to track progress
    eval_steps=100,
    logging_steps=5,
    save_steps=100,
    
    # Keep the best model
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    
    # Performance optimizations
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    fp16=True,
    
    # Generation settings
    eval_strategy="steps",
    predict_with_generate=True,
    generation_max_length=225,
    
    # Other settings
    report_to=["tensorboard"],
    push_to_hub=True,
    remove_unused_columns=True,
)


**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
print(training_args.output_dir)

### Training

Training will take approximately 5-10 hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB.
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/home/ubuntu/Downloads/drone-test-model")

Our best WER is 32.0% - not bad for 8h of training data! We can make our model more accessible on the Hub with appropriate tags and README information.
You can change these values to match your dataset, language and model
name accordingly:

In [ ]:
kwargs = {
    # "dataset_tags": "mozilla-foundation/common_voice_19_0",
    "dataset": "drone command tsv6",  # a 'pretty' name for the training dataset
    "dataset_args": "config: en, split: test",
    "language": "en",
    "model_name": "Drone test En - Kai Shan",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}

The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**kwargs)

## Building a Demo

Now that we've fine-tuned our model we can build a demo to show
off its ASR capabilities! We'll make use of 🤗 Transformers
`pipeline`, which will take care of the entire ASR pipeline,
right from pre-processing the audio inputs to decoding the
model predictions.

Running the example below will generate a Gradio demo where we
can record speech through the microphone of our computer and input it to
our fine-tuned Whisper model to transcribe the corresponding text:

In [ ]:
# from transformers import pipeline
# import gradio as gr

# # Load from your local model directory instead of the Hub
# pipe = pipeline(
#     model="/home/ubuntu/Downloads/drone-small-en",  # Use your local model path
#     task="automatic-speech-recognition"
# )

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe,
#     inputs=gr.Audio(sources="microphone", type="filepath"),
#     outputs="text",
#     title="Whisper Small English",  # Changed to reflect English model
#     description="Realtime demo for English speech recognition using a fine-tuned Whisper small model.",
# )

# iface.launch()

from transformers import pipeline, WhisperForConditionalGeneration, WhisperProcessor
import torch
import gradio as gr
import os

def load_local_model(model_path):
    """
    Load a local Whisper model from the specified path
    """
    if not os.path.exists(model_path):
        raise ValueError(f"Model path {model_path} does not exist")
        
    # Load the model configuration and processor from the local directory
    processor = WhisperProcessor.from_pretrained(model_path)
    model = WhisperForConditionalGeneration.from_pretrained(model_path)
    
    # Create the pipeline with the local model and processor
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
    )
    
    return pipe

# Path to your local model
MODEL_PATH = "/home/ubuntu/Downloads/drone_small_en"

try:
    # Load the local model
    pipe = load_local_model(MODEL_PATH)
    
    def transcribe(audio):
        if audio is None:
            return ""
        text = pipe(audio)["text"]
        return text

    # Create Gradio interface
    iface = gr.Interface(
        fn=transcribe,
        inputs=gr.Audio(sources="microphone", type="filepath"),
        outputs="text",
        title="Local Whisper Model Demo",
        description="Speech recognition using locally stored Whisper model"
    )

    if __name__ == "__main__":
        iface.launch()
        
except Exception as e:
    print(f"Error loading model: {str(e)}")

# from transformers import pipeline, WhisperForConditionalGeneration, WhisperProcessor
# import torch
# import gradio as gr
# import os
# import numpy as np
# import queue
# import threading
# import time

# def load_local_model(model_path):
#     """
#     Load a local Whisper model from the specified path
#     """
#     if not os.path.exists(model_path):
#         raise ValueError(f"Model path {model_path} does not exist")
    
#     processor = WhisperProcessor.from_pretrained(model_path)
#     model = WhisperForConditionalGeneration.from_pretrained(model_path)
    
#     pipe = pipeline(
#         "automatic-speech-recognition",
#         model=model,
#         tokenizer=processor.tokenizer,
#         feature_extractor=processor.feature_extractor,
#         chunk_length_s=30,
#         stride_length_s=5
#     )
#     return pipe

# class AudioTranscriber:
#     def __init__(self, pipe):
#         self.pipe = pipe
#         self.audio_queue = queue.Queue()
#         self.is_running = False
#         self.current_text = ""
    
#     def process_audio(self, audio_chunk, sample_rate):
#         try:
#             if len(audio_chunk.shape) > 1:
#                 audio_chunk = audio_chunk.mean(axis=1)
            
#             audio_chunk = audio_chunk / np.max(np.abs(audio_chunk))
            
#             result = self.pipe({"sampling_rate": sample_rate, "raw": audio_chunk})
#             return result["text"].strip()
#         except Exception as e:
#             print(f"Error processing audio: {str(e)}")
#             return ""

#     def start_processing(self):
#         self.is_running = True
#         threading.Thread(target=self._process_queue, daemon=True).start()

#     def stop_processing(self):
#         self.is_running = False
#         self.current_text = ""

#     def _process_queue(self):
#         while self.is_running:
#             try:
#                 audio_data = self.audio_queue.get(timeout=1)
#                 text = self.process_audio(audio_data[0], audio_data[1])
#                 if text:
#                     self.current_text += " " + text
#             except queue.Empty:
#                 continue
#             except Exception as e:
#                 print(f"Error in processing queue: {str(e)}")

# def create_stream_interface(model_path):
#     try:
#         pipe = load_local_model(model_path)
#         transcriber = AudioTranscriber(pipe)
        
#         def stream_audio(audio_chunk, state):
#             if audio_chunk is None:
#                 return state
            
#             transcriber.audio_queue.put((audio_chunk[1], audio_chunk[0]))
#             return transcriber.current_text
        
#         def start_streaming():
#             transcriber.start_processing()
#             return ""
        
#         def stop_streaming():
#             transcriber.stop_processing()
#             final_text = transcriber.current_text
#             transcriber.current_text = ""
#             return final_text

#         # Create Gradio interface
#         with gr.Blocks() as iface:
#             gr.Markdown("# Real-time Speech Recognition")
            
#             with gr.Row():
#                 start_btn = gr.Button("Start Recording")
#                 stop_btn = gr.Button("Stop Recording")
            
#             output_text = gr.Textbox(
#                 label="Transcription",
#                 placeholder="Transcription will appear here...",
#                 lines=5
#             )
            
#             # Updated Audio component syntax
#             audio_input = gr.Audio(
#                 sources=["microphone"],  # Changed from source to sources
#                 streaming=True,
#                 type="numpy"
#             )
            
#             start_btn.click(
#                 fn=start_streaming,
#                 outputs=[output_text]
#             )
            
#             stop_btn.click(
#                 fn=stop_streaming,
#                 outputs=[output_text]
#             )
            
#             audio_input.stream(
#                 fn=stream_audio,
#                 inputs=[audio_input, output_text],
#                 outputs=[output_text],
#                 show_progress=False
#             )

#         return iface

#     except Exception as e:
#         print(f"Error setting up interface: {str(e)}")
#         return None

# # Path to your local model
# MODEL_PATH = "/home/ubuntu/Downloads/drone_small_en"

# if __name__ == "__main__":
#     iface = create_stream_interface(MODEL_PATH)
#     if iface:
#         iface.launch()
#     else:
#         print("Failed to create interface")

In [ ]:
!pip install onnx onnxruntime transformers[onnx]

In [ ]:
!pip list | grep torch

In [ ]:
!/bin/python3 -m transformers.onnx --model="/home/ubuntu/Downloads/whisper-small-en" --opset=14 --feature='speech2seq-lm' "/home/ubuntu/Downloads/whisper-small-en.onnx"

convert binary to pt (1)

In [ ]:
# from transformers import WhisperForConditionalGeneration
# import torch
# import os

# save_dir = "/home/ubuntu/Downloads/drone_small_en"
# weights_path = os.path.join(save_dir, "whisper_weights.pt")

# # Ensure the save directory exists
# os.makedirs(save_dir, exist_ok=True)

# # Check if the model is already saved locally
# if os.path.exists(os.path.join(save_dir, "pytorch_model.bin")):
#     print("Loading model from local directory...")
#     model = WhisperForConditionalGeneration.from_pretrained(save_dir)
# else:
#     print("Downloading model from Hugging Face (first-time use)...")
#     model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small.en")
#     model.save_pretrained(save_dir)  # Save it for future use
#     print(f"Model saved locally at: {save_dir}")

# # Save only the model weights in a .pt file
# torch.save(model.state_dict(), weights_path)
# print(f"Model weights saved in PyTorch format at: {weights_path}")

from transformers import WhisperForConditionalGeneration
import torch
import os
import glob

def print_file_size(filepath):
    """Print file size in MB"""
    size_bytes = os.path.getsize(filepath)
    size_mb = size_bytes / (1024 * 1024)
    print(f"{filepath}: {size_mb:.2f} MB")

# Define paths
save_dir = "/home/ubuntu/Downloads/drone_small_en"
weights_path = os.path.join(save_dir, "whisper_weights.pt")

# List and print sizes of all relevant files
print("\nChecking files in directory:")
for file in glob.glob(os.path.join(save_dir, "*")):
    print_file_size(file)

# Load your .bin file and inspect its content
bin_files = glob.glob(os.path.join(save_dir, "*.bin"))
if bin_files:
    print("\nAttempting to load .bin file:")
    for bin_file in bin_files:
        print(f"\nInspecting {bin_file}")
        try:
            content = torch.load(bin_file)
            print(f"Content type: {type(content)}")
            if hasattr(content, '__dict__'):
                print("Attributes:", content.__dict__.keys())
        except Exception as e:
            print(f"Error loading {bin_file}: {str(e)}")

# Load base model for size comparison
print("\nLoading base Whisper-small model for comparison:")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small.en")
torch.save(base_model.state_dict(), "temp_base_weights.pt")
print_file_size("temp_base_weights.pt")
os.remove("temp_base_weights.pt")  # Cleanup

# Print model sizes
print("\nModel size comparisons:")
print(f"Your .bin file: {os.path.getsize(bin_files[0]) / (1024 * 1024):.2f} MB")
print(f"Generated .pt file: {os.path.getsize(weights_path) / (1024 * 1024):.2f} MB")
print(f"Expected Whisper-small size: ~460-480 MB")


convert binary to pt

In [ ]:
#!/bin/env python3
import whisper
import re
import torch

def hf_to_whisper_states(text):
    text = re.sub('.layers.', '.blocks.', text)
    text = re.sub('.self_attn.', '.attn.', text)
    text = re.sub('.q_proj.', '.query.', text)
    text = re.sub('.k_proj.', '.key.', text)
    text = re.sub('.v_proj.', '.value.', text)
    text = re.sub('.out_proj.', '.out.', text)
    text = re.sub('.fc1.', '.mlp.0.', text)
    text = re.sub('.fc2.', '.mlp.2.', text)
    text = re.sub('.fc3.', '.mlp.3.', text)
    text = re.sub('.fc3.', '.mlp.3.', text)
    text = re.sub('.encoder_attn.', '.cross_attn.', text)
    text = re.sub('.cross_attn.ln.', '.cross_attn_ln.', text)
    text = re.sub('.embed_positions.weight', '.positional_embedding', text)
    text = re.sub('.embed_tokens.', '.token_embedding.', text)
    text = re.sub('model.', '', text)
    text = re.sub('attn.layer_norm.', 'attn_ln.', text)
    text = re.sub('.final_layer_norm.', '.mlp_ln.', text)
    text = re.sub('encoder.layer_norm.', 'encoder.ln_post.', text)
    text = re.sub('decoder.layer_norm.', 'decoder.ln.', text)
    text = re.sub('proj_out.weight', 'decoder.token_embedding.weight', text)
    return text

# Load HF Model
hf_state_dict = torch.load("training_args.bin", map_location=torch.device('cpu'))    # pytorch_model.bin file

# Rename layers
for key in list(hf_state_dict.keys())[:]:
    new_key = hf_to_whisper_states(key)
    hf_state_dict[new_key] = hf_state_dict.pop(key)

model = whisper.load_model('medium')
dims = model.dims
# Save it
torch.save({
    "dims": model.dims.__dict__,
    "model_state_dict": hf_state_dict
}, "training_args.bin")



#!/bin/env python3
# import whisper
# import re
# import torch
# from transformers import AutoModelForSpeechSeq2Seq

# def hf_to_whisper_states(text):
#     text = re.sub(r'\.layers\.', '.blocks.', text)
#     text = re.sub(r'\.self_attn\.', '.attn.', text)
#     text = re.sub(r'\.q_proj\.', '.query.', text)
#     text = re.sub(r'\.k_proj\.', '.key.', text)
#     text = re.sub(r'\.v_proj\.', '.value.', text)
#     text = re.sub(r'\.out_proj\.', '.out.', text)
#     text = re.sub(r'\.fc1\.', '.mlp.0.', text)
#     text = re.sub(r'\.fc2\.', '.mlp.2.', text)
#     text = re.sub(r'\.fc3\.', '.mlp.3.', text)
#     text = re.sub(r'\.encoder_attn\.', '.cross_attn.', text)
#     text = re.sub(r'\.cross_attn\.ln\.', '.cross_attn_ln.', text)
#     text = re.sub(r'\.embed_positions\.weight', '.positional_embedding', text)
#     text = re.sub(r'\.embed_tokens\.', '.token_embedding.', text)
#     text = re.sub(r'model\.', '', text)
#     text = re.sub(r'attn\.layer_norm\.', 'attn_ln.', text)
#     text = re.sub(r'\.final_layer_norm\.', '.mlp_ln.', text)
#     text = re.sub(r'encoder\.layer_norm\.', 'encoder.ln_post.', text)
#     text = re.sub(r'decoder\.layer_norm\.', 'decoder.ln.', text)
#     text = re.sub(r'proj_out\.weight', 'decoder.token_embedding.weight', text)
#     return text

# def convert_hf_to_whisper(input_model_path, output_model_path):
#     try:
#         # Load the HuggingFace model
#         model = AutoModelForSpeechSeq2Seq.from_pretrained(input_model_path)
#         hf_state_dict = model.state_dict()
        
#         # Rename layers
#         whisper_state_dict = {}
#         for key in list(hf_state_dict.keys()):
#             new_key = hf_to_whisper_states(key)
#             whisper_state_dict[new_key] = hf_state_dict[key]
        
#         # Load base whisper model to get dimensions
#         base_model = whisper.load_model('small')
        
#         # Save in Whisper format
#         torch.save({
#             "dims": base_model.dims.__dict__,
#             "model_state_dict": whisper_state_dict,
#             "audio_encoder_engine": hf_state_dict.get("audio_encoder_engine", {})
#         }, output_model_path)
        
#         print(f"Model successfully converted and saved to {output_model_path}")
#         return True
    
#     except Exception as e:
#         print(f"Error during conversion: {str(e)}")
#         return False

# if __name__ == "__main__":
#     input_model_path = "/home/ubuntu/Downloads/drone_small_en"  # Path to your fine-tuned model
#     output_model_path = "drone_small_en.pt"  # The output .pt file
#     convert_hf_to_whisper(input_model_path, output_model_path)




Convert pt to pth which is not need for whisper_trt. 

In [ ]:
import torch
import os

def convert_model(input_path, output_path):
    # First verify the input file exists
    if not os.path.exists(input_path):
        print(f"Error: Input file '{input_path}' does not exist!")
        print(f"Current working directory is: {os.getcwd()}")
        return False

    try:
        # Method 1: Direct load and save
        model = torch.load(input_path)
        torch.save(model, output_path)
        print(f"Successfully converted using direct load/save method")
        return True
    except Exception as e:
        print(f"Direct conversion failed: {str(e)}")
        try:
            # Method 2: Load with map_location
            model = torch.load(input_path, map_location=torch.device('cpu'))
            torch.save(model, output_path)
            print(f"Successfully converted using map_location method")
            return True
        except Exception as e:
            print(f"map_location conversion failed: {str(e)}")
            try:
                # Method 3: Load state dict
                checkpoint = torch.load(input_path)
                if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
                    torch.save(checkpoint['state_dict'], output_path)
                    print(f"Successfully converted state dict")
                    return True
                else:
                    print("No state_dict found in checkpoint")
                    return False
            except Exception as e:
                print(f"State dict conversion failed: {str(e)}")
                return False

# Usage example
if __name__ == "__main__":
    # Use the full path to your PT file
    input_file = "/home/ubuntu/Documents/ws_ros2/src/whisper_trt/drone_small_en.pt"
    output_file = "/home/ubuntu/Documents/ws_ros2/src/whisper_trt/drone_small_en_trt.pth"
    
    print(f"Converting {input_file} to {output_file}")
    success = convert_model(input_file, output_file)
    
    if success:
        # Verify the conversion
        try:
            loaded_model = torch.load(output_file)
            print("Conversion successful - new model loads correctly")
        except Exception as e:
            print(f"Conversion verification failed: {str(e)}")

## Closing Remarks

In this blog, we covered a step-by-step guide on fine-tuning Whisper for multilingual ASR
using 🤗 Datasets, Transformers and the Hugging Face Hub. For more details on the Whisper model, the Common Voice dataset and the theory behind fine-tuning, refere to the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper). If you're interested in fine-tuning other
Transformers models, both for English and multilingual ASR, be sure to check out the
examples scripts at [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).